In [9]:
# import os
# # import torch
# from astropy.io import fits
import numpy as np
# from models.network_unet import UNetAuthor, UNet, UNetRes, UNetRes2, UNetRes3, UNetRes4
# from models.basic_models import simple_CNN
# from models.models_helper import get_model
# from lightning_denoiser import GradMatch
# from skimage import io
import glob

In [6]:
path = '/work/sc004/sc004/tc1213/data/bio_images/testingset_true_tiff/'
files = glob.glob(path + '*.tif')
len(files)

20

In [14]:
a = np.arange(100)
a[0:-1:10]

array([ 0, 10, 20, 30, 40, 50, 60, 70, 80, 90])

In [5]:
start = 7000
end = start+250 if start+250 <= len(files) else len(files)
end

7115

In [14]:
get_model

<function models.models_helper.get_model(architecture, n_ch=1, n_ch_in=None, checkpoint_pth=None)>

In [73]:
'''
Test stability of a denoiser by iteratively denoising a given noisy image
'''
'/work/sc004/sc004/tc1213/data/bio_images/trainingset_true_tiff'

print('Loading model...')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device = {}'.format(device))
model = UNetRes4(nc=[32, 64, 128, 256], nb=2)
network_pth = '/work/sc004/sc004/tc1213/Prox-Pnp2/GS_denoising/logs_UNetRes4_astro/1-1/final.ckpt'
# model, net_name, clip_val, _ = get_model('DnCNN_nobn', n_ch=1, checkpoint_pth=network_pth)
# network_pth = '/work/sc004/sc004/tc1213/Prox-Pnp2/GS_denoising/logs_UNetRes2/s4-4_3/final.ckpt'
checkpoint = torch.load(network_pth, map_location=device)      
# print(checkpoint)      
new_state_dict = {}
# for key, val in checkpoint.items():
for key, val in checkpoint['state_dict'].items():
    new_key = key.split('student_grad.model.')[-1]
    new_state_dict[new_key] = val
model.load_state_dict(new_state_dict, strict=True)
# model = model
# model.load_state_dict(checkpoint['state_dict'])
print('Network loaded.')

path = '/work/sc004/sc004/tc1213/data/bio_images/trainingset_true_tiff/'
files = glob.glob(path + '*.tif')
print(len(files))
skip = len(files)

taskname = '3.2e-4'

for i, file in enumerate(files):
    if (i+1)%np.floor(len(files)/skip) == 0:
        # print(i+1)
        img = io.imread(file) 
        # print(torch.Tensor(img).unsqueeze(0).size())
        # Nx, Ny = img.shape    
        img_tmp = img + np.random.normal(size=img.shape) * 0.00032
        for _ in range(500):
            img_denoised = model(torch.Tensor(img_tmp).unsqueeze(0).unsqueeze(0))
        # print(img_denoised.squeeze(0).size())
        # print([img.shape,img_denoised.detach().numpy().size])
        img_combined = np.column_stack((img, img_denoised.squeeze(0).squeeze(0).detach().numpy()))
        outpth = '/work/sc004/sc004/tc1213/data/bio_images/trainingset_true_tiff_' + taskname
        # print(os.path.exists(outpth))
        if (i+1) == np.floor(len(files)/skip):
            if not os.path.exists(outpth):
                os.makedirs(outpth, exist_ok=True)
        # print(os.path.basename(img))
        io.imsave(outpth + '/' + os.path.basename(file), img_combined)
        # print(img_combined.shape)
        # io.imshow(img_combined)
        

# gt_pth = '/work/sc004/sc004/tc1213/data/data_groundtruth/3c353.fits'
# img_gt = fits.getdata(gt_pth)
# print('Loaded groundtruth image.')


Loading model...
device = cpu
Network loaded.
7115


KeyboardInterrupt: 

In [24]:
print('Loading model...')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device = {}'.format(device))
model = UNetRes3(nc=[32, 64, 128, 256], nb=2)
network_pth = '/work/sc004/sc004/tc1213/Prox-Pnp2/GS_denoising/logs_UNetRes2/s4-4_3/final.ckpt'
checkpoint = torch.load(network_pth, map_location=device)            
new_state_dict = {}
for key, val in checkpoint['state_dict'].items():
    new_key = key.split('student_grad.model.')[-1]
    new_state_dict[new_key] = val
model.load_state_dict(new_state_dict, strict=True)
model = model
# model.load_state_dict(checkpoint['state_dict'])
print('Network loaded.')


Loading model...
device = cpu
Network loaded.


In [7]:
model

UNetRes3(
  (m_head): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (m_down1): Sequential(
    (0): ResBlock(
      (res): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): ReLU(inplace=True)
        (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
    )
    (1): ResBlock(
      (res): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): ReLU(inplace=True)
        (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
    )
    (2): Conv2d(32, 64, kernel_size=(2, 2), stride=(2, 2), bias=False)
  )
  (m_down2): Sequential(
    (0): ResBlock(
      (res): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): ReLU(inplace=True)
        (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), pa

In [30]:
noise = 3.2e-4
img_noisy = np.random.normal(0, noise, img_gt.shape) + img_gt
x = torch.from_numpy(img_noisy).unsqueeze(0).unsqueeze(0).float()
result_pth = '/work/sc004/sc004/tc1213/results/bio/denoiser/dncnn/bio2/' + str(noise)
os.makedirs(result_pth, exist_ok = True)
fits.writeto(result_pth + '/noisy.fits', x.detach().numpy(), overwrite=True)
print('Created noisy image and saved at {}.'.format(result_pth))

for i in range(4000):
    # print(i)
    x = model.forward(x)
    if (i+1)%100 == 0:
        fits.writeto(result_pth + '/result_{}.fits'.format(str(i+1)), x.detach().numpy(), overwrite=True)
    

print('Finished denoising and results saved at {}.'.format(result_pth))



Created noisy image and saved at /work/sc004/sc004/tc1213/results/bio/denoiser/dncnn/bio2/0.00032.


: 

: 

In [27]:
x.unsqueeze(0).size()

torch.Size([1, 1, 512, 512])

In [ ]:
x